In [1]:
import surprise
from surprise import Dataset
from surprise import Reader
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from functools import partial

In [2]:
min_max_log_df = pd.read_csv("min_max_log_scaled.csv")

In [3]:
min_max_log_df

,program_name,config,cycles
0,raytrace,"4,64k,1m,16m",0.641700
1,raytrace,"4,32k,1m,16m",0.838413
2,raytrace,"4,32k,2m,32m",1.000000
3,raytrace,"4,64k,2m,32m",0.949523
4,swaptions,"4,64k,1m,32m",0.398326
5,swaptions,"4,32k,2m,32m",0.420648
6,swaptions,"4,64k,2m,32m",0.397875
7,swaptions,"8,32k,2m,32m",0.301434
8,swaptions,"4,32k,1m,32m",0.420648
9,swaptions,"4,64k,1m,16m",0.398326


In [4]:
%load_ext cython

In [5]:
%%cython

from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

cimport numpy as np  # noqa
import numpy as np
from six.moves import range
from surprise.utils import get_rng
from surprise.prediction_algorithms.algo_base import AlgoBase
from surprise.prediction_algorithms.predictions import PredictionImpossible

class NMF(AlgoBase):
    """A collaborative filtering algorithm based on Non-negative Matrix
    Factorization.
    This algorithm is very similar to :class:`SVD`. The prediction
    :math:`\\hat{r}_{ui}` is set as:
    .. math::
        \hat{r}_{ui} = q_i^Tp_u,
    where user and item factors are kept **positive**. Our implementation
    follows that suggested in :cite:`NMF:2014`, which is equivalent to
    :cite:`Zhang96` in its non-regularized form. Both are direct applications
    of NMF for dense matrices :cite:`NMF_algo`.
    The optimization procedure is a (regularized) stochastic gradient descent
    with a specific choice of step size that ensures non-negativity of factors,
    provided that their initial values are also positive.
    At each step of the SGD procedure, the factors :math:`f` or user :math:`u`
    and item :math:`i` are updated as follows:
    .. math::
        p_{uf} &\\leftarrow p_{uf} &\cdot \\frac{\\sum_{i \in I_u} q_{if}
        \\cdot r_{ui}}{\\sum_{i \in I_u} q_{if} \\cdot \\hat{r_{ui}} +
        \\lambda_u |I_u| p_{uf}}\\\\
        q_{if} &\\leftarrow q_{if} &\cdot \\frac{\\sum_{u \in U_i} p_{uf}
        \\cdot r_{ui}}{\\sum_{u \in U_i} p_{uf} \\cdot \\hat{r_{ui}} +
        \lambda_i |U_i| q_{if}}\\\\
    where :math:`\lambda_u` and :math:`\lambda_i` are regularization
    parameters.
    This algorithm is highly dependent on initial values. User and item factors
    are uniformly initialized between ``init_low`` and ``init_high``. Change
    them at your own risks!
    A biased version is available by setting the ``biased`` parameter to
    ``True``. In this case, the prediction is set as
    .. math::
        \hat{r}_{ui} = \mu + b_u + b_i + q_i^Tp_u,
    still ensuring positive factors. Baselines are optimized in the same way as
    in the :class:`SVD` algorithm. While yielding better accuracy, the biased
    version seems highly prone to overfitting so you may want to reduce the
    number of factors (or increase regularization).
    Args:
        n_factors: The number of factors. Default is ``15``.
        n_epochs: The number of iteration of the SGD procedure. Default is
            ``50``.
        biased(bool): Whether to use baselines (or biases). Default is
            ``False``.
        reg_pu: The regularization term for users :math:`\lambda_u`. Default is
            ``0.06``.
        reg_qi: The regularization term for items :math:`\lambda_i`. Default is
            ``0.06``.
        reg_bu: The regularization term for :math:`b_u`. Only relevant for
            biased version. Default is ``0.02``.
        reg_bi: The regularization term for :math:`b_i`. Only relevant for
            biased version. Default is ``0.02``.
        lr_bu: The learning rate for :math:`b_u`. Only relevant for biased
            version. Default is ``0.005``.
        lr_bi: The learning rate for :math:`b_i`. Only relevant for biased
            version. Default is ``0.005``.
        init_low: Lower bound for random initialization of factors. Must be
            greater than ``0`` to ensure non-negative factors. Default is
            ``0``.
        init_high: Higher bound for random initialization of factors. Default
            is ``1``.
        random_state(int, RandomState instance from numpy, or ``None``):
            Determines the RNG that will be used for initialization. If
            int, ``random_state`` will be used as a seed for a new RNG. This is
            useful to get the same initialization over multiple calls to
            ``fit()``.  If RandomState instance, this same instance is used as
            RNG. If ``None``, the current RNG from numpy is used.  Default is
            ``None``.
        verbose: If ``True``, prints the current epoch. Default is ``False``.
    Attributes:
        pu(numpy array of size (n_users, n_factors)): The user factors (only
            exists if ``fit()`` has been called)
        qi(numpy array of size (n_items, n_factors)): The item factors (only
            exists if ``fit()`` has been called)
        bu(numpy array of size (n_users)): The user biases (only
            exists if ``fit()`` has been called)
        bi(numpy array of size (n_items)): The item biases (only
            exists if ``fit()`` has been called)
    """

    def __init__(self, n_factors=15, n_epochs=50, biased=False, reg_pu=.06,
                 reg_qi=.06, reg_bu=.02, reg_bi=.02, lr_bu=.005, lr_bi=.005,
                 init_low=0, init_high=1, random_state=None, verbose=False):

        self.n_factors = n_factors
        self.n_epochs = n_epochs
        self.biased = biased
        self.reg_pu = reg_pu
        self.reg_qi = reg_qi
        self.lr_bu = lr_bu
        self.lr_bi = lr_bi
        self.reg_bu = reg_bu
        self.reg_bi = reg_bi
        self.init_low = init_low
        self.init_high = init_high
        self.random_state = random_state
        self.verbose = verbose

        if self.init_low < 0:
            raise ValueError('init_low should be greater than zero')

        AlgoBase.__init__(self)

    def fit(self, trainset):

        AlgoBase.fit(self, trainset)
        self.sgd(trainset)

        return self

    def sgd(self, trainset):

        # user and item factors
        cdef np.ndarray[np.double_t, ndim=2] pu
        cdef np.ndarray[np.double_t, ndim=2] qi

        # user and item biases
        cdef np.ndarray[np.double_t] bu
        cdef np.ndarray[np.double_t] bi

        # auxiliary matrices used in optimization process
        cdef np.ndarray[np.double_t, ndim=2] user_num
        cdef np.ndarray[np.double_t, ndim=2] user_denom
        cdef np.ndarray[np.double_t, ndim=2] item_num
        cdef np.ndarray[np.double_t, ndim=2] item_denom

        cdef int u, i, f
        cdef double r, est, l, dot, err
        cdef double reg_pu = self.reg_pu
        cdef double reg_qi = self.reg_qi
        cdef double reg_bu = self.reg_bu
        cdef double reg_bi = self.reg_bi
        cdef double lr_bu = self.lr_bu
        cdef double lr_bi = self.lr_bi
        cdef double global_mean = self.trainset.global_mean

        # Randomly initialize user and item factors
        rng = get_rng(self.random_state)
        pu = rng.uniform(self.init_low, self.init_high,
                         size=(trainset.n_users, self.n_factors))
        qi = rng.uniform(self.init_low, self.init_high,
                         size=(trainset.n_items, self.n_factors))

        bu = np.zeros(trainset.n_users, np.double)
        bi = np.zeros(trainset.n_items, np.double)

        if not self.biased:
            global_mean = 0

        for current_epoch in range(self.n_epochs):

            if self.verbose:
                print("Processing epoch {}".format(current_epoch))

            # (re)initialize nums and denoms to zero
            user_num = np.zeros((trainset.n_users, self.n_factors))
            user_denom = np.zeros((trainset.n_users, self.n_factors))
            item_num = np.zeros((trainset.n_items, self.n_factors))
            item_denom = np.zeros((trainset.n_items, self.n_factors))

            # Compute numerators and denominators for users and items factors
            for u, i, r in trainset.all_ratings():

                # compute current estimation and error
                dot = 0  # <q_i, p_u>
                for f in range(self.n_factors):
                    dot += qi[i, f] * pu[u, f]
                est = global_mean + bu[u] + bi[i] + dot
                err = r - est

                # update biases
                if self.biased:
                    bu[u] += lr_bu * (err - reg_bu * bu[u])
                    bi[i] += lr_bi * (err - reg_bi * bi[i])

                # compute numerators and denominators
                for f in range(self.n_factors):
                    user_num[u, f] += qi[i, f] * r
                    user_denom[u, f] += qi[i, f] * est
                    item_num[i, f] += pu[u, f] * r
                    item_denom[i, f] += pu[u, f] * est

            # Update user factors
            for u in trainset.all_users():
                n_ratings = len(trainset.ur[u])
                for f in range(self.n_factors):
                    user_denom[u, f] += n_ratings * reg_pu * pu[u, f]
                    if user_denom[u,f] != 0: # check if not 0 to prevent div by 0
                        pu[u, f] *= user_num[u, f] / user_denom[u, f]

            # Update item factors
            for i in trainset.all_items():
                n_ratings = len(trainset.ir[i])
                for f in range(self.n_factors):
                    item_denom[i, f] += n_ratings * reg_qi * qi[i, f]
                    if item_denom[i,f] != 0: # check if not 0 to prevent div by 0
                        qi[i, f] *= item_num[i, f] / item_denom[i, f]

        self.bu = bu
        self.bi = bi
        self.pu = pu
        self.qi = qi

    def estimate(self, u, i):
        # Should we cythonize this as well?

        known_user = self.trainset.knows_user(u)
        known_item = self.trainset.knows_item(i)

        if self.biased:
            est = self.trainset.global_mean

            if known_user:
                est += self.bu[u]

            if known_item:
                est += self.bi[i]

            if known_user and known_item:
                est += np.dot(self.qi[i], self.pu[u])

        else:
            if known_user and known_item:
                est = np.dot(self.qi[i], self.pu[u])
            else:
                raise PredictionImpossible('User and item are unknown.')

        return est

In [6]:
#'epochs': 20, 'n_factors': 3, 'lr_bu': 0.03, 'lr_bi': 0.01, 'reg_pu': 0.01, 'reg_qi': 0.001, 'reg_bu': 0.005, 
# 'reg_bi': 0.05
best_NMF = NMF(biased=True, n_epochs=20, n_factors=3, lr_bu=0.03, lr_bi=0.01, 
                        reg_pu=0.01, reg_qi=0.001, reg_bu=0.005, reg_bi=0.05)

In [7]:
# {'n_epochs': 35, 'n_factors': 2, 'lr_all': 0.55, 'reg_all': 0.005}
best_SVD = surprise.SVD(n_epochs=35, n_factors=2, lr_all=0.55, reg_all=0.005)

In [8]:
train_set = min_max_log_df[min_max_log_df.program_name != "fluidanimate"]
train_set

,program_name,config,cycles
0,raytrace,"4,64k,1m,16m",0.641700
1,raytrace,"4,32k,1m,16m",0.838413
2,raytrace,"4,32k,2m,32m",1.000000
3,raytrace,"4,64k,2m,32m",0.949523
4,swaptions,"4,64k,1m,32m",0.398326
5,swaptions,"4,32k,2m,32m",0.420648
6,swaptions,"4,64k,2m,32m",0.397875
7,swaptions,"8,32k,2m,32m",0.301434
8,swaptions,"4,32k,1m,32m",0.420648
9,swaptions,"4,64k,1m,16m",0.398326


fluidanimate is our test set so it will not be touched. One record will become our unknown program run

In [9]:
unknown_program = min_max_log_df[min_max_log_df.program_name == "fluidanimate"].iloc[0:1]
unknown_program

,program_name,config,cycles
35,fluidanimate,"4,32k,1m,16m",0.603548


In [10]:
ground_truth = min_max_log_df[min_max_log_df.program_name == "fluidanimate"].iloc[1:3]
ground_truth

,program_name,config,cycles
36,fluidanimate,"4,64k,2m,32m",0.688139
37,fluidanimate,"4,32k,2m,32m",0.688416


In [11]:
excluded_train_set = train_set[train_set.program_name != 'swaptions']
excluded_train_set = pd.concat([excluded_train_set, unknown_program])
excluded_train_set

,program_name,config,cycles
0,raytrace,"4,64k,1m,16m",0.641700
1,raytrace,"4,32k,1m,16m",0.838413
2,raytrace,"4,32k,2m,32m",1.000000
3,raytrace,"4,64k,2m,32m",0.949523
13,blackscholes,"4,32k,2m,16m",0.026172
14,blackscholes,"8,32k,1m,16m",0.000182
15,blackscholes,"4,64k,1m,16m",0.026020
16,blackscholes,"8,64k,2m,16m",0.000000
17,blackscholes,"4,32k,1m,16m",0.026161
18,blackscholes,"8,32k,2m,16m",0.000182


In [12]:
not_excluded_train_set = pd.concat([train_set, unknown_program])
not_excluded_train_set

,program_name,config,cycles
0,raytrace,"4,64k,1m,16m",0.641700
1,raytrace,"4,32k,1m,16m",0.838413
2,raytrace,"4,32k,2m,32m",1.000000
3,raytrace,"4,64k,2m,32m",0.949523
4,swaptions,"4,64k,1m,32m",0.398326
5,swaptions,"4,32k,2m,32m",0.420648
6,swaptions,"4,64k,2m,32m",0.397875
7,swaptions,"8,32k,2m,32m",0.301434
8,swaptions,"4,32k,1m,32m",0.420648
9,swaptions,"4,64k,1m,16m",0.398326


In [13]:
reader = Reader(rating_scale=(0, 1))
excluded_data = Dataset.load_from_df(excluded_train_set, reader)

In [14]:
reader = Reader(rating_scale=(0, 1))
not_excluded_data = Dataset.load_from_df(not_excluded_train_set, reader)

In [15]:
best_NMF_excluded = best_NMF.fit(excluded_data.build_full_trainset())

In [16]:
estimates_NMF_excluded = {}
for possible_config in excluded_train_set.config.unique():
    est = best_NMF_excluded.predict(uid="fluidanimate", iid=possible_config).est
    estimates_NMF_excluded[possible_config] = est

estimates_NMF_excluded

{'4,64k,1m,16m': 0.44262831416347437,
 '4,32k,1m,16m': 0.5981106225524454,
 '4,32k,2m,32m': 0.6088224244206739,
 '4,64k,2m,32m': 0.7385659322883482,
 '4,32k,2m,16m': 0.3520515341568772,
 '8,32k,1m,16m': 0.30932011275999344,
 '8,64k,2m,16m': 0.3013282028856193,
 '8,32k,2m,16m': 0.29858316345753655,
 '4,64k,2m,16m': 0.3540130354707291,
 '8,64k,1m,16m': 0.3017642669156712,
 '8,32k,1m,32m': 0.30932635258266855,
 '4,64k,1m,32m': 0.361658399147473,
 '8,64k,2m,32m': 0.3035314159595445,
 '4,32k,1m,32m': 0.7213680597029141,
 '8,32k,2m,32m': 0.30950322997200586,
 '8,64k,1m,32m': 0.30363297420785934}

In [17]:
best_SVD_excluded = best_SVD.fit(excluded_data.build_full_trainset())

In [18]:
estimates_SVD_excluded = {}
for possible_config in excluded_train_set.config.unique():
    est = best_SVD_excluded.predict(uid="fluidanimate", iid=possible_config).est
    estimates_SVD_excluded[possible_config] = est

estimates_SVD_excluded

{'4,64k,1m,16m': 0.48496373448496044,
 '4,32k,1m,16m': 0.6031619860579865,
 '4,32k,2m,32m': 0.6099337992455199,
 '4,64k,2m,32m': 0.6009225783734523,
 '4,32k,2m,16m': 0.49614361022000053,
 '8,32k,1m,16m': 0.4983996438220082,
 '8,64k,2m,16m': 0.49742089925048794,
 '8,32k,2m,16m': 0.5288328401433312,
 '4,64k,2m,16m': 0.5156892223069781,
 '8,64k,1m,16m': 0.5196470675087168,
 '8,32k,1m,32m': 0.540157134522747,
 '4,64k,1m,32m': 0.41341448686157223,
 '8,64k,2m,32m': 0.4946938707037289,
 '4,32k,1m,32m': 0.591613850609702,
 '8,32k,2m,32m': 0.5182804246672665,
 '8,64k,1m,32m': 0.5146379433551311}

In [19]:
best_NMF_not_excluded = best_NMF.fit(not_excluded_data.build_full_trainset())

In [20]:
estimates_NMF_not_excluded = {}
for possible_config in not_excluded_train_set.config.unique():
    est = best_NMF_not_excluded.predict(uid="fluidanimate", iid=possible_config).est
    estimates_NMF_not_excluded[possible_config] = est

estimates_NMF_not_excluded

{'4,64k,1m,16m': 0.5041657368527588,
 '4,32k,1m,16m': 0.6024119516041082,
 '4,32k,2m,32m': 0.6055536446742118,
 '4,64k,2m,32m': 0.6331521798950932,
 '4,64k,1m,32m': 0.4557094095506974,
 '8,32k,2m,32m': 0.451140446263285,
 '4,32k,1m,32m': 0.6314456452174639,
 '4,32k,2m,16m': 0.6116233824723392,
 '4,64k,2m,16m': 0.5888859646515812,
 '8,32k,1m,16m': 0.3346740924200817,
 '8,64k,2m,16m': 0.33181229904977494,
 '8,32k,2m,16m': 0.33041228040660053,
 '8,64k,1m,16m': 0.3307369951789306,
 '8,32k,1m,32m': 0.3314402842924508,
 '8,64k,2m,32m': 0.33312071508317526,
 '8,64k,1m,32m': 0.3343023870416856}

In [21]:
best_SVD_not_excluded = best_SVD.fit(not_excluded_data.build_full_trainset())

In [22]:
estimates_SVD_not_excluded = {}
for possible_config in not_excluded_train_set.config.unique():
    est = best_SVD_not_excluded.predict(uid="fluidanimate", iid=possible_config).est
    estimates_SVD_not_excluded[possible_config] = est

estimates_SVD_not_excluded

{'4,64k,1m,16m': 0.4815187778655484,
 '4,32k,1m,16m': 0.6031631029474878,
 '4,32k,2m,32m': 0.622017221347426,
 '4,64k,2m,32m': 0.6017021859022875,
 '4,64k,1m,32m': 0.41821198906266915,
 '8,32k,2m,32m': 0.5253816300694624,
 '4,32k,1m,32m': 0.6083814022947561,
 '4,32k,2m,16m': 0.5236688294068265,
 '4,64k,2m,16m': 0.5278407766212959,
 '8,32k,1m,16m': 0.5134223663225255,
 '8,64k,2m,16m': 0.5031980956615357,
 '8,32k,2m,16m': 0.508944216503413,
 '8,64k,1m,16m': 0.4924206213736542,
 '8,32k,1m,32m': 0.5217355557734136,
 '8,64k,2m,32m': 0.49844906280013035,
 '8,64k,1m,32m': 0.4972056612399338}

In [23]:
NMF_Excluded_ground_preds = [estimates_NMF_excluded[i] for i in ground_truth.config.values]
RMSE_NMF_Excluded= mean_squared_error(ground_truth.cycles,NMF_Excluded_ground_preds, squared=False)

SVD_Excluded_ground_preds = [estimates_SVD_excluded[i] for i in ground_truth.config.values]
RMSE_SVD_Excluded= mean_squared_error(ground_truth.cycles,SVD_Excluded_ground_preds, squared=False)

NMF_Not_Excluded_ground_preds = [estimates_NMF_not_excluded[i] for i in ground_truth.config.values]
RMSE_NMF_Not_Excluded= mean_squared_error(ground_truth.cycles,NMF_Not_Excluded_ground_preds, squared=False)

SVD_Not_Excluded_ground_preds = [estimates_SVD_not_excluded[i] for i in ground_truth.config.values]
RMSE_SVD_Not_Excluded= mean_squared_error(ground_truth.cycles,SVD_Not_Excluded_ground_preds, squared=False)

In [24]:
print("NMF Excluded:", RMSE_NMF_Excluded)
print("SVD Excluded:", RMSE_SVD_Excluded)
print("NMF Not Excluded:", RMSE_NMF_Not_Excluded)
print("SVD Not Excluded:", RMSE_SVD_Not_Excluded)

NMF Excluded: 0.06662570392271754
SVD Excluded: 0.08296456373278875
NMF Not Excluded: 0.07031987456336802
SVD Not Excluded: 0.07707203607905003
